# Data Challenge 4b

## Download data

In [11]:
#! pip install -q lalsuite
#! pip install -q gwpy
#! pip install -q pycbc
# -- Click "restart runtime" in the runtime menu

# -- download data
! wget https://www.gw-openscience.org/s/workshop3/challenge/challenge3.gwf

--2022-05-27 16:14:16--  https://www.gw-openscience.org/s/workshop3/challenge/challenge3.gwf
Resolving www.gw-openscience.org (www.gw-openscience.org)... 131.215.113.73
Connecting to www.gw-openscience.org (www.gw-openscience.org)|131.215.113.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256815066 (245M)
Saving to: ‘challenge3.gwf’

challenge3.gwf      100%[===================>] 244.92M  8.49MB/s    in 38s     

2022-05-27 16:14:55 (6.43 MB/s) - ‘challenge3.gwf’ saved [256815066/256815066]



## Simple-To-Use matched-filtering code from sample

In [15]:
from pycbc import frame
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.waveform import get_fd_waveform
from pycbc.filter import matched_filter
from pycbc.vetoes import power_chisq
from pycbc.events.ranking import newsnr

import numpy

import pylab
%matplotlib inline

def challenge_matched_filter(file_name,channel_name,mass):
    print("Looking at file {} with template mass {} M_sol".format(file_name,mass))
    #start = 0
    #end = start + 128
    #ts = read_frame(file_name, channel_name, start, end)
    ts = frame.read_frame(file_name, channel_name)
    ts = highpass(ts, 15.0)
    strain_ts = resample_to_delta_t(ts, 1.0/2048)
    conditioned_ts = strain_ts.crop(2, 2)
    psd_ts = conditioned_ts.psd(16)
    psd_ts = interpolate(psd_ts, conditioned_ts.delta_f)
    psd_ts = inverse_spectrum_truncation(psd_ts, int(4 * conditioned_ts.sample_rate),low_frequency_cutoff=15)
    
    hp_x, _ = get_fd_waveform(approximant="IMRPhenomD",
                                mass1=mass, mass2=mass,
                                f_lower=20.0, delta_f=conditioned_ts.delta_f)
    hp_x.resize(len(psd_ts))

    # For each observatory use this template to calculate the SNR time series
    snr_x = matched_filter(hp_x, conditioned_ts, psd=psd_ts, low_frequency_cutoff=20).crop(5, 4)

    #pylab.figure(figsize=[14, 4])
    #pylab.plot(snr_x.sample_times, abs(snr_x), label='H1')
    #pylab.title('SNR Time Series')
    #pylab.grid()
    #pylab.xlabel('Time (s)')
    #pylab.ylabel('Signal-to-noise (SNR)')
    #pylab.show()
    
    #peak_x = numpy.argmax(abs(snr_x))
    #snr_x_peak = snr_x[peak_x]
    #time_x = snr_x.sample_times[peak_x]
    #print("We found a possible signal at {}s with SNR {}".format(time_x, abs(snr_x_peak)))

    chisq_x = {}
    nbins = 26
    chisq_x = power_chisq(hp_x, conditioned_ts, nbins, psd_ts, low_frequency_cutoff=20.0)
    chisq_x = chisq_x.crop(5, 4)

    dof_x = nbins * 2 - 2
    chisq_x /= dof_x


    # The rho-hat term above is named "newsnr" here
    nsnr_x = newsnr(abs(snr_x), chisq_x)

    peak_x = numpy.argmax(abs(nsnr_x))
    nsnr_x_peak = nsnr_x[peak_x]
    time_x = snr_x.sample_times[peak_x]
    
    nsnr_out = nsnr_x_peak
    time_out = time_x
    
    #print("We found a signal at {}s with (reweighed) SNR {}".format(time_x, abs(nsnr_x_peak)))
    #Plot the new SNR timeseries
    #pylab.figure(figsize=[14, 4])
    #pylab.plot(snr_x.sample_times, nsnr_x, label='H1')
    #pylab.title('NewSNR Timeseries')
    #pylab.grid()
    #pylab.xlabel('Time (s)')
    #pylab.ylabel('Re-weighted Signal-to-noise')
    #pylab.show()
    
    return nsnr_out, time_out

In [16]:
file = 'challenge3.gwf'
channel = 'H1:CHALLENGE3'
#channel = 'L1:CHALLENGE3'

### Using loop to search for mass

In [17]:
import numpy
snr_threshold = 8.0
n = 21
m = []
s = []
t = []
for mass in numpy.array([20/(n-1)*i+10 for i in range(n)]):
    nsnr_x_peak_out, time_x_out = challenge_matched_filter(file,channel, mass)
    if nsnr_x_peak_out >= snr_threshold:
        m.append(mass)
        s.append(nsnr_x_peak_out)
        t.append(time_x_out)
        print("SNR {} > {} at mass {}".format(nsnr_x_peak_out, snr_threshold, mass))

Looking at file challenge3.gwf with template mass 10.0 M_sol
SNR 26.870953418426545 > 8.0 at mass 10.0
Looking at file challenge3.gwf with template mass 11.0 M_sol
Looking at file challenge3.gwf with template mass 12.0 M_sol
Looking at file challenge3.gwf with template mass 13.0 M_sol
Looking at file challenge3.gwf with template mass 14.0 M_sol
Looking at file challenge3.gwf with template mass 15.0 M_sol
Looking at file challenge3.gwf with template mass 16.0 M_sol
Looking at file challenge3.gwf with template mass 17.0 M_sol
SNR 32.38142857790615 > 8.0 at mass 17.0
Looking at file challenge3.gwf with template mass 18.0 M_sol
Looking at file challenge3.gwf with template mass 19.0 M_sol
Looking at file challenge3.gwf with template mass 20.0 M_sol
Looking at file challenge3.gwf with template mass 21.0 M_sol
SNR 11.0859236000081 > 8.0 at mass 21.0
Looking at file challenge3.gwf with template mass 22.0 M_sol
SNR 39.69172100034723 > 8.0 at mass 22.0
Looking at file challenge3.gwf with templat

In [18]:
import numpy
snr_threshold = 10.0
n = 20
for mass in numpy.array([19/(n-1)*i+31 for i in range(n)]):
    nsnr_x_peak_out, time_x_out = challenge_matched_filter(file,channel, mass)
    if nsnr_x_peak_out >= snr_threshold:
        m.append(mass)
        s.append(nsnr_x_peak_out)
        t.append(time_x_out)
        print("SNR {} > {} at mass {}".format(nsnr_x_peak_out, snr_threshold, mass))

In [19]:
print(m)
print(s)
print(t)

[10.0, 17.0, 21.0, 22.0, 23.0, 24.0, 25.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0]
[26.870953418426545, 32.38142857790615, 11.0859236000081, 39.69172100034723, 14.367890590956746, 9.691856195582984, 8.314740860974318, 8.353597248588102, 10.019865172415294, 10.44077077391879, 10.843135009764767, 10.89038959639953, 12.822202694508045, 13.68167751336103, 13.578986328998441, 12.53917872915822, 11.858658874651082, 11.105951253620534, 10.358208274114766]
[2483.96875, 1638.158203125, 1204.7109375, 1204.71044921875, 1204.70947265625, 1204.708984375, 1204.70849609375, 2995.34423828125, 2995.34375, 2995.34326171875, 2995.34228515625, 3319.2548828125, 3319.25439453125, 3319.25390625, 3319.25341796875, 3319.25341796875, 3319.2529296875, 3319.25244140625, 3319.251953125]


In [ ]:
import numpy
snr_threshold = 8.0
channel = 'L1:CHALLENGE3'
n = 21
m = []
s = []
t = []
for mass in numpy.array([20/(n-1)*i+10 for i in range(n)]):
    nsnr_x_peak_out, time_x_out = challenge_matched_filter(file,channel, mass)
    if nsnr_x_peak_out >= snr_threshold:
        m.append(mass)
        s.append(nsnr_x_peak_out)
        t.append(time_x_out)
        print("SNR {} > {} at mass {}".format(nsnr_x_peak_out, snr_threshold, mass))

In [ ]:
import numpy
snr_threshold = 10.0
n = 20
for mass in numpy.array([19/(n-1)*i+31 for i in range(n)]):
    nsnr_x_peak_out, time_x_out = challenge_matched_filter(file,channel, mass)
    if nsnr_x_peak_out >= snr_threshold:
        m.append(mass)
        s.append(nsnr_x_peak_out)
        t.append(time_x_out)
        print("SNR {} > {} at mass {}".format(nsnr_x_peak_out, snr_threshold, mass))